In [201]:
import re
import json
import pickle

from sklearn.metrics import classification_report

In [202]:
with open("/Utilisateurs/umushtaq/am_reasoning/saved_models/pe_pipeline_prompt2_Meta-Llama-3.1-8B-Instruct-bnb-4bit/pe_pipeline_results_2.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [203]:
grounds = results['grounds']
predictions = results['predictions']

In [204]:
grounds

['### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\']}\n{"relation_types": [((8, 4), \'S\'), ((7, 4), \'S\'), ((6, 4), \'S\'), ((10, 11), \'S\'), ((9, 5), \'S\'), ((11, 5), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'C\']}\n{"relation_types": [((6, 3), \'S\'), ((7, 4), \'A\'), ((8, 7), \'A\'), ((9, 4), \'S\'), ((10, 5), \'S\'), ((11, 5), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'C\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\']}\n{"relation_types": [((7, 4), \'S\'), ((8, 9), \'S\'), ((10, 9), \'S\'), ((9, 4), \'S\'), ((11, 5), \'S\'), ((12, 5), \'S\'), ((13, 6), \'S\'), ((14, 6), \'S\'), ((15, 6), \'S\'), ((16, 6), \'S\'), ((17, 16), \'S\'), ((18, 16), \'S\'), ((19, 16), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'P\', \'P\', \'C\', 

In [205]:
predictions

['### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\']}\n{"relation_types": [((7, 4), \'S\'), ((8, 4), \'S\'), ((6, 4), \'S\'), ((10, 5), \'S\'), ((11, 5), \'S\'), ((9, 5), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'C\']}\n{"relation_types": [((6, 3), \'S\'), ((7, 4), \'S\'), ((8, 4), \'S\'), ((9, 4), \'S\'), ((11, 10), \'S\'), ((10, 5), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'P\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\']}\n{"relation_types": [((7, 4), \'S\'), ((8, 4), \'S\'), ((9, 4), \'S\'), ((10, 4), \'S\'), ((11, 5), \'S\'), ((12, 5), \'S\'), ((13, 6), \'S\'), ((14, 6), \'S\'), ((15, 6), \'S\'), ((16, 6), \'S\'), ((17, 6), \'S\'), ((18, 6), \'S\'), ((19, 6), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'P\', \'P\', \'C\', \'P\', \'C

### Process Grounds

In [ ]:
def process_grounds(grounds):
    
    arg_types = []
    #rel_pairs = []
    rel_types = []
    
    for grounds in grounds:
        
        args = grounds.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', grounds.split("\n")[2])
        rels = eval(rels)["relation_types"]
        rels = [(tuple(pair), label) for pair, label in rels]
        rel_types.append(rels)
        
    return arg_types, rel_types

In [239]:
grounds_acc, grounds_aric  = process_grounds(grounds)

In [241]:
len(grounds_acc), len(grounds_aric)

(80, 80)

### Process Predictions

In [242]:
def process_predictions(predictions):
    
    arg_types = []
    rel_types = []
    
    for pred in predictions:
        
        args = pred.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', pred.split("\n")[2])
        rels = eval(rels)["relation_types"]
        rels = [(tuple(pair), label) for pair, label in rels]
        rel_types.append(rels)
        
    return arg_types, rel_types

In [243]:
predictions_acc, predictions_aric  = process_predictions(predictions)

In [245]:
len(predictions_acc), len(predictions_aric)

(80, 80)

### Compute ACC metrics

In [247]:
grounds = [elem for sublist in grounds_acc for elem in sublist]
predictions = [elem for sublist in predictions_acc for elem in sublist]

In [248]:
print(classification_report(grounds, predictions, digits=3))

ValueError: Found input variables with inconsistent numbers of samples: [1266, 1268]

### Compute ARI metrics

In [249]:
len(grounds_aric), len(predictions_aric)

(80, 80)

In [250]:
grounds_aric[0]

[((8, 4), 'S'),
 ((7, 4), 'S'),
 ((6, 4), 'S'),
 ((10, 11), 'S'),
 ((9, 5), 'S'),
 ((11, 5), 'S')]

In [251]:
predictions_aric[0]

[((7, 4), 'S'),
 ((8, 4), 'S'),
 ((6, 4), 'S'),
 ((10, 5), 'S'),
 ((11, 5), 'S'),
 ((9, 5), 'S')]

In [252]:
nr_acs = [len(elem) for elem in grounds_acc]

In [286]:
def build_grounds(grounds, n_acs):
    
    ground_pairs = [elem[0] for elem in grounds]
    all_triples = [(i, j, "NR") for i in range(1, n_acs+1) for j in range(1, n_acs+1) if (i != j and (i, j) not in ground_pairs)]
    
    grounds_t = [(elem[0][0], elem[0][1], elem[1]) for elem in grounds]
    
    return all_triples + grounds_t

In [291]:
grounds_triples = []

for ground, n_acs in zip(grounds_aric, nr_acs):
    
    grounds_triples.append(build_grounds(ground, n_acs))

In [292]:
len(grounds_triples)

80

In [296]:
def build_predictions(ground, pred, n_acs):
    
    predictions = []
    
    for p in pred:
        
        if p in ground:
            predictions.append(p)
    
    prediction_pairs = [elem[0] for elem in pred]
    all_triples = [(i, j, "NR") for i in range(1, n_acs+1) for j in range(1, n_acs+1) if (i != j and (i, j) not in prediction_pairs)]
    
    preds_t = [(elem[0][0], elem[0][1], elem[1]) for elem in predictions]
    
    return all_triples + preds_t

In [299]:
prediction_triples = []

for ground, pred, n_acs in zip(grounds_aric, predictions_aric, nr_acs):
    
    prediction_triples.append(build_predictions(ground, pred, n_acs))

In [300]:
len(prediction_triples)

80

In [43]:
grounds = [elem for sublist in grounds_l for elem in sublist]
predictions = [elem for sublist in predictions_l for elem in sublist]

In [44]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

          NR      0.980     0.980     0.980     19386
           R      0.514     0.507     0.511       806

    accuracy                          0.961     20192
   macro avg      0.747     0.744     0.745     20192
weighted avg      0.961     0.961     0.961     20192



### Compute ARC metrics

In [ ]:
grounds_ari

[['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'A', 'A', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'A', 'S', 'S'],
 ['S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A', 'A', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'A', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S

In [106]:
grounds_ari

[[(8, 4), (7, 4), (6, 4), (10, 11), (9, 5), (11, 5)],
 [(6, 3), (7, 4), (8, 7), (9, 4), (10, 5), (11, 5)],
 [(7, 4),
  (8, 9),
  (10, 9),
  (9, 4),
  (11, 5),
  (12, 5),
  (13, 6),
  (14, 6),
  (15, 6),
  (16, 6),
  (17, 16),
  (18, 16),
  (19, 16)],
 [(5, 3),
  (4, 3),
  (9, 6),
  (11, 13),
  (12, 13),
  (10, 6),
  (13, 6),
  (14, 7),
  (15, 7),
  (18, 8),
  (17, 16),
  (16, 8)],
 [(6, 4), (7, 4), (8, 7), (9, 7), (10, 5), (11, 5)],
 [(5, 2),
  (6, 2),
  (7, 2),
  (8, 2),
  (9, 2),
  (10, 2),
  (11, 4),
  (4, 3),
  (12, 13),
  (13, 3)],
 [(11, 10),
  (12, 10),
  (7, 4),
  (8, 4),
  (9, 4),
  (10, 4),
  (13, 5),
  (14, 5),
  (15, 5),
  (16, 5),
  (17, 6),
  (18, 6),
  (19, 6),
  (20, 6),
  (21, 6)],
 [(6, 4), (8, 9), (7, 9), (9, 5), (10, 5), (11, 5)],
 [(6, 3), (7, 4), (8, 4), (9, 5)],
 [(5, 4), (6, 4), (7, 4), (8, 4), (11, 9), (12, 13), (9, 13)],
 [(8, 3),
  (9, 3),
  (10, 3),
  (11, 3),
  (12, 13),
  (13, 4),
  (14, 4),
  (6, 5),
  (7, 5)],
 [(4, 3), (6, 7), (7, 5), (9, 10)],
 [(9, 3)

In [107]:
aric_grounds = []
    
for rel_pairs, rel_types in zip(grounds_ari, grounds_arc):
    
    result = list(zip(rel_pairs, rel_types))
    aric_grounds.append(result)

In [108]:
len(aric_grounds)

80

In [109]:
aric_grounds[0]

[((8, 4), 'S'),
 ((7, 4), 'S'),
 ((6, 4), 'S'),
 ((10, 11), 'S'),
 ((9, 5), 'S'),
 ((11, 5), 'S')]

In [110]:
aric_predictions = []
    
for rel_pairs, rel_types in zip(predictions_ari, predictions_arc):
        
    result = list(zip(rel_pairs, rel_types))
    aric_predictions.append(result)

In [111]:
len(aric_predictions)

80

In [112]:
aric_predictions[0]

[((7, 4), 'S'), ((8, 4), 'S'), ((10, 5), 'S'), ((11, 5), 'S'), ((6, 4), 'S')]

In [172]:
len(nr_acs)

80

In [175]:
len(aric_grounds), len(aric_predictions)

(80, 80)

In [176]:
aric_grounds[0]

[((8, 4), 'S'),
 ((7, 4), 'S'),
 ((6, 4), 'S'),
 ((10, 11), 'S'),
 ((9, 5), 'S'),
 ((11, 5), 'S')]

In [177]:
aric_predictions[0]

[((7, 4), 'S'), ((8, 4), 'S'), ((10, 5), 'S'), ((11, 5), 'S'), ((6, 4), 'S')]

In [178]:
g_0 = set(aric_grounds[0])

In [179]:
g_0

{((6, 4), 'S'),
 ((7, 4), 'S'),
 ((8, 4), 'S'),
 ((9, 5), 'S'),
 ((10, 11), 'S'),
 ((11, 5), 'S')}

In [180]:
p_0 = set(aric_predictions[0])

In [181]:
p_0

{((6, 4), 'S'), ((7, 4), 'S'), ((8, 4), 'S'), ((10, 5), 'S'), ((11, 5), 'S')}

In [182]:
z = g_0.intersection(p_0)

In [183]:
z

{((6, 4), 'S'), ((7, 4), 'S'), ((8, 4), 'S'), ((11, 5), 'S')}

In [121]:
correct_predictions = []
correct_grounds = []

for aric_ground, aric_pred in zip(aric_grounds, aric_predictions):
    
    common_elements = [item for item in aric_pred if item in aric_ground]
            
    correct_predictions.append(common_elements)
    correct_grounds.append(aric_ground)

In [122]:
len(correct_predictions), len(correct_grounds)

(80, 80)

In [123]:
correct_predictions[0]

[((7, 4), 'S'), ((8, 4), 'S'), ((11, 5), 'S'), ((6, 4), 'S')]

In [124]:
correct_grounds[0]

[((8, 4), 'S'),
 ((7, 4), 'S'),
 ((6, 4), 'S'),
 ((10, 11), 'S'),
 ((9, 5), 'S'),
 ((11, 5), 'S')]

In [128]:
ground_types = [[item[1] for item in sublist] for sublist in correct_grounds]

In [130]:
len(ground_types)

80

In [131]:
prediction_types = [[item[1] for item in sublist] for sublist in correct_predictions]

In [132]:
len(prediction_types)

80

In [173]:
ground_types

[['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'A', 'A', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'A', 'S', 'S'],
 ['S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A', 'A', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'A', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S

In [174]:
prediction_types

[['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S'],
 ['S', 'S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S'],
 ['S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S'],
 ['S', 'S'],
 ['S', 'S', 'S', 'S', 'A'],
 ['S', 'S'],
 ['S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S'],
 ['S', 'S', 'S'],
 ['S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S'],
 ['S', 'S'],
 ['S'],
 ['S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S'],
 ['S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'A', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A'],
 [],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S

In [187]:
bad_idx = []
oppo_d = {"S": "A", "A": "S"}

for idx, (i,j) in enumerate(zip(ground_types, prediction_types)):
    
    if len(i) > len(j):
        prediction_types[idx] = prediction_types[idx] + [oppo_d[elem] for elem in ground_types[idx][len(j):]]
        
    elif len(j) > len(i):        
        prediction_types[idx] = prediction_types[idx][:len(i)]
        #print(idx, len(i), len(j))
        #bad_idx.append(idx)

In [188]:
grounds = [elem for sublist in ground_types for elem in sublist]
predictions = [elem for sublist in prediction_types for elem in sublist]

In [189]:
len(grounds), len(predictions)

(809, 809)

In [190]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

           A      0.005     0.048     0.009        42
           S      0.904     0.493     0.638       767

    accuracy                          0.470       809
   macro avg      0.455     0.270     0.324       809
weighted avg      0.858     0.470     0.605       809



In [ ]:
### Maybe it makes no sense to do ari and arc separately.